In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [14]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [5]:
len(tsla)

1551

In [6]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046089,94.574012,91.246200,91.745842,6840800
2019-01-03,92.009804,93.555870,91.849543,93.329618,6346000
2019-01-04,95.054810,95.752423,93.065662,93.254204,7269100
2019-01-07,97.204224,97.826417,95.460187,95.507318,6263100
2019-01-08,98.278931,98.806854,96.883697,98.212940,9391000


In [21]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [24]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [26]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954758,227.889151,223.117450,223.144664,126925200,NaN,NaN
2019-01-03,221.539017,225.494261,221.049139,225.185814,144140700,NaN,NaN
2019-01-04,228.959610,229.612771,224.224203,224.605211,142628800,NaN,NaN
2019-01-07,230.764816,232.189059,228.324538,229.231704,103139100,NaN,NaN
2019-01-08,232.932953,233.422830,230.420114,232.978328,102512600,NaN,NaN
2019-01-09,234.021576,234.874314,232.406822,233.649635,95006600,NaN,NaN
2019-01-10,234.847153,235.101158,231.780927,232.470382,96823900,NaN,NaN
2019-01-11,234.937805,234.965019,233.168821,233.758473,73858100,NaN,NaN


In [114]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)


                                               title  \
0  New Inflation Data Dismays Bulls, CoreWeave IP...   
1  Assessing Apple's Performance Against Competit...   
2  Apple's Options Frenzy: What You Need to Know ...   
3  Trading The SPY As PCE Inflation Report Takes ...   
4  EU To Issue Minimal Fines To Apple And Meta To...   

                                                 url   time_published  \
0  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
1  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   
2  https://www.benzinga.com/insights/options/25/0...  20250328T134533   
3  https://www.benzinga.com/markets/equities/25/0...  20250328T125805   
4  https://www.benzinga.com/markets/25/03/4453457...  20250328T122729   

               authors                                            summary  \
0   [The Arora Report]  To gain an edge, this is what you need to know...   
1  [Benzinga Insights]  Amidst the fast-paced and highly competitive b

In [115]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
1,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
2,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
3,Trading The SPY As PCE Inflation Report Takes ...,https://www.benzinga.com/markets/equities/25/0...,20250328T125805,[RIPS],Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.084859,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0813..."
4,EU To Issue Minimal Fines To Apple And Meta To...,https://www.benzinga.com/markets/25/03/4453457...,20250328T122729,[Namrata Sen],The European Union ( EU ) is reportedly set to...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.103777,Neutral,"[{'ticker': 'META', 'relevance_score': '0.2658..."
5,3 Northern Mutual Funds for Solid Returns,https://www.zacks.com/stock/news/2436613/3-nor...,20250328T111100,[Zacks Equity Research],"Invest in Northern mutual funds like NOIEX, NM...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.272886,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1496..."
6,2 Breakout Growth Stocks You Can Buy and Hold ...,https://www.fool.com/investing/2025/03/28/2-br...,20250328T110700,[Anders Bylund],Looking for undervalued stocks with explosive ...,https://g.foolcdn.com/editorial/images/812691/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.298788,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.1478..."
7,Intel's SuperFluid Cooling Tech 'Possibly Suit...,https://www.benzinga.com/25/03/44532855/intels...,20250328T104410,[Rishabh Mishra],Intel Corp. INTC SuperFluid cooling tech could...,https://editorial-assets.benzinga.com/wp-conte...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.226797,Somewhat-Bullish,"[{'ticker': 'NVDA', 'relevance_score': '0.4078..."
8,Should Strive 500 ETF ( STRV ) Be on Your In...,https://www.zacks.com/stock/news/2436589/shoul...,20250328T102007,[Zacks Equity Research],Style Box ETF report for ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.183048,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1331..."
9,Should Invesco Dividend Achievers ETF ( PFM )...,https://www.zacks.com/stock/news/2436592/shoul...,20250328T102005,[Zacks Equity Research],Style Box ETF report for ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financi